<a href="https://colab.research.google.com/github/WebberMark02/machine-learning-project/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importo le librerie necessarie e scelgo di utilizzare "tensorflow" come
backend per "Keras"

In [ ]:
import numpy as np
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import keras
from matplotlib import pyplot as plt
import sklearn

Imposto le variabili globali

In [ ]:
# TODO: Incrementa il numero di campioni di training e il numero di campioni di validation.
# Per capire a quanto impostarli, calcola il numero massimo di possibili combinazioni di due immagini nel
# training set di Cifar10 e il numero massimo di possibili combinazioni di due immagini nel validation set di 10000
# immagini ottenuto dal training set di Cifar10.

seed = 42
training_set_size = 200000
testing_set_size = 10000
validation_set_size = 20000
image_shape = (32, 32, 3)
batch_size = 256
epochs = 250
learning_rate = 0.001
l2 = 0.0
early_stopping_patience = 10

Definisco il generatore di immagini sulle quali il modello verrà addestrato e testato.
Il generatore restituisce "batchsize" immagini; ogni immagine è la media di due immagini
scelte casualmente rispettivamente da X1 e X2.
Il generatore restituisce, inoltre, una coppia di matrici, che possiamo denotare con y.
y[0][i] e y[1][i] sono, rispettivamente, la classe della prima componente e la classe della seconda componente
della immagine i-esima.
Ogni classe è rappresentata con un vettore di lunghezza 5 dove ogni elemento è nullo tranne quello
il cui indice corrisponde alla classe stessa.

In [ ]:
def datagenerator(x1,x2,y1,y2,batchsize):
  # Numero di immagini in X1
  size1 = x1.shape[0]
  # Numero di immagini in X2
  size2 = x2.shape[0]

  # Trasforma, per esempio, [0, 1, 2] in [[1, 0, 0], [0, 1, 0], [0, 0, 1]]
  y1_cat = keras.utils.to_categorical(y1, num_classes=5)
  y2_cat = keras.utils.to_categorical(y2-5, num_classes=5)

  while True:
    num1 = np.random.randint(0, size1, batchsize)
    num2 = np.random.randint(0, size2, batchsize)
    x_data = (x1[num1] + x2[num2]) / 2.0
    y_data = [y1_cat[num1],y2_cat[num2]]

    yield x_data, y_data

Definisco una funzione per il controllo del bilanciamento delle classi.
Mi servirà per verificare che la divisione stratificata del training set
di Cifar10 in training set e validation set abbia avuto successo.

In [ ]:
def stampa_percentuale( y, title="" ):
  if title:print(title)
  # Calcoliamo le occorrenze di ciascuna classe nel dataset
  unique, counts = np.unique(y, return_counts=True)
  percentuali = (counts / len(y)) * 100 # calcolo la % di occorrenze per ciascuna classe
  # Stampiamo le occorrenze e le percentuali
  for classe, conteggio, percentuale in zip(unique, counts, percentuali):
      print(f"Classe {classe}: Occorrenze = {conteggio}, Percentuale {percentuale} %" )
  print(f"Totale occorrenze : {sum(counts)}")
  print()

Definisco un modello casuale.
Mi servirà per valutare che la rete abbia prestazioni migliori di quelle casuali.

In [ ]:
def random_model(x):
  #the random model ingnore the input x and return a pair of random classes
  return(np.random.randint(0,5,(10000,2)))

Ora ha inizio la fase di caricamento e pre-processing dei dataset che verranno utilizzati
per addestrare e esaminare le prestazioni della rete.

Prima di tutto, carico il training set e il testing set di Cifar10

In [ ]:
(cifar10_x_train, cifar10_y_train), (cifar10_x_test, cifar10_y_test) = keras.datasets.cifar10.load_data()
assert cifar10_x_train.shape == (50000, 32, 32, 3)
assert cifar10_x_test.shape == (10000, 32, 32, 3)
assert cifar10_y_train.shape == (50000, 1)
assert cifar10_y_test.shape == (10000, 1)

In [ ]:
# Tutte le possibili classi di Cifar10
classes = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

Normalizzo i valori dei pixel delle immagini di Cifar10 nell'intervallo [0,1] così da poterle dare in input
alla rete neurale che costruirò

In [ ]:
cifar10_x_train = (cifar10_x_train/255.).astype(np.float32)
cifar10_x_test = (cifar10_x_test/255.).astype(np.float32)

Controllo il bilanciamento delle classi nel training set prima della divisione stratificata

In [ ]:
stampa_percentuale(cifar10_y_train, 'Training set completo')

Divido il training set di Cifar10 in due insiemi: il training set e il validation set.
Quest'ultimo verrà usato per l'ottimizzazione degli iper-parametri della rete.
Il validation set conterrà il 20% delle immagini del training set di Cifar10.
Uso la stratificazione per mantenere le classi nelle stesse proporzioni.

In [ ]:
cifar10_x_train, cifar10_x_val, cifar10_y_train, cifar10_y_val = sklearn.model_selection.train_test_split(cifar10_x_train, cifar10_y_train, test_size=0.2, stratify=cifar10_y_train, random_state=seed)

Controllo il bilanciamento delle classi nel training set e nel validation set ottenuti dalla divisione stratificata

In [ ]:
stampa_percentuale(cifar10_y_train, 'Training set risultante')
stampa_percentuale(cifar10_y_val, 'Validation set risultante')

Divido le immagini in due gruppi: un gruppo per le etichette ["airplane", "automobile", "bird", "cat", "deer"]
e un secondo gruppo per le etichette ["dog", "frog", "horse", "ship", "truck"]

In [ ]:
cond_1 = cifar10_y_train[:,0] < 5
cifar10_x_train_1 = cifar10_x_train[cond_1]
cifar10_y_train_1 = cifar10_y_train[cond_1]

cond_2 = cifar10_y_train[:,0] >= 5
cifar10_x_train_2 = cifar10_x_train[cond_2]
cifar10_y_train_2 = cifar10_y_train[cond_2]

cond_1_test = cifar10_y_test[:,0] < 5
cifar10_x_test_1 = cifar10_x_test[cond_1_test]
cifar10_y_test_1 = cifar10_y_test[cond_1_test]

cond_2_test = cifar10_y_test[:,0] >= 5
cifar10_x_test_2 = cifar10_x_test[cond_2_test]
cifar10_y_test_2 = cifar10_y_test[cond_2_test]

cond_1_val = cifar10_y_val[:,0] < 5
cifar10_x_val_1 = cifar10_x_val[cond_1_val]
cifar10_y_val_1 = cifar10_y_val[cond_1_val]

cond_2_val = cifar10_y_val[:,0] >= 5
cifar10_x_val_2 = cifar10_x_val[cond_2_val]
cifar10_y_val_2 = cifar10_y_val[cond_2_val]

Creo tre istanze del generatore per generare il training set, il validation set e il testing set finali.

In [ ]:
traingen = datagenerator(cifar10_x_train_1, cifar10_x_train_2, cifar10_y_train_1, cifar10_y_train_2, training_set_size)
valgen = datagenerator(cifar10_x_val_1, cifar10_x_val_2, cifar10_y_val_1, cifar10_y_val_2, validation_set_size)
testgen = datagenerator(cifar10_x_test_1, cifar10_x_test_2, cifar10_y_test_1, cifar10_y_test_2, testing_set_size)

Creo il training set e il validation set.

In [ ]:
x_train, y_train = next(traingen)
x_val, y_val = next(valgen)

Verifico che le dimensioni dei due dataset siano corrette

In [ ]:
print(x_train.shape)
print(x_val.shape)

print(len(y_train))
print(len(y_val))

print(y_train[0].shape)
print(y_train[1].shape)
print(y_val[0].shape)
print(y_val[1].shape)

Controllo che i valori dei pixel delle immagini appartengano all'intervallo [0,1]

In [ ]:
print(np.min(x_train[0]), np.max(x_train[0]))
print(np.min(x_val[0]), np.max(x_val[0]))

La fase di preparazione dei dataset è completa.
Ora viene definita la rete neurale.

In [ ]:
def build_neural_network():
  inputs = keras.Input(shape = image_shape, name = 'InputImage')
  conv = keras.layers.Conv2D(
                            filters=3,                    # The number of filters
                            kernel_size=(5, 5),           # Height and width of the 2D Convolution window.
                            strides=(1, 1),               # (Height,width)
                            padding="valid",              # "same" per renderlo della stessa dimensione dell'output
                            dilation_rate=(1, 1),         # serve ad allargare il campo visivo del kernel
                            groups=1,                     # channels e filtri devono essere divisibili per il numero dei gruppi, gruppi non possono essere piu grandi dei channels
                            activation='relu',
                            kernel_regularizer = keras.regularizers.l2(l2=l2),
                            bias_regularizer = keras.regularizers.l2(l2=l2),
                          )(inputs)
  pool = keras.layers.MaxPooling2D(pool_size = (2, 2))(conv)
  flatten = keras.layers.Flatten()(pool)
  dense = keras.layers.Dense(units = 100,
                               activation = 'softmax',
                               kernel_regularizer = keras.regularizers.l2(l2=l2),
                               bias_regularizer = keras.regularizers.l2(l2=l2)
                               )(flatten)
  output1 = keras.layers.Dense(units = 5,
                               activation = 'softmax',
                               name = 'FirstComponentClass',
                               kernel_regularizer = keras.regularizers.l2(l2=l2),
                               bias_regularizer = keras.regularizers.l2(l2=l2)
                               )(dense)
  output2 = keras.layers.Dense(5,
                               activation = 'softmax',
                               name = 'SecondComponentClass',
                               kernel_regularizer = keras.regularizers.l2(l2=l2),
                               bias_regularizer = keras.regularizers.l2(l2=l2)
                               )(dense)
  model = keras.Model(inputs = inputs, outputs = [output1, output2], name = "Cifar10_Avg_Model")
  return model

Istanzio la rete neurale e mostro i suoi dettagli

In [ ]:
model = build_neural_network()
model.summary()

Visualizzo un plot della rete

In [ ]:
keras.utils.plot_model(model, "model.png", show_shapes = True, show_layer_names = True)

Compilo la rete.

In [ ]:
model.compile(
    loss=['categorical_crossentropy', 'categorical_crossentropy'],
    optimizer=keras.optimizers.AdamW(learning_rate = learning_rate),
    metrics=['accuracy', 'accuracy'],
)

Definisco una callback EarlyStopping.
Essa valuta alla fine di ogni epoca la loss sul validation set
e decide se fermare l'addestramento oppure no.
Utile per stabilire automaticamente un buon numero di epoche di
addestramento della rete.

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',   # Monitora la loss sulla metrica indicata
    min_delta=0.001,      # Variazione minima da considerare come miglioramento
    patience=early_stopping_patience,          # Numero di epoche senza miglioramenti prima di fermare l'addestramento
    mode='auto',          # oppure prende "min","max", seleziona la direzione in automatico
    restore_best_weights=True,  # Ripristina i pesi migliori quando l'addestramento si ferma
    start_from_epoch=5    # Inizia il monitoraggio dall'epoca 5
)

Addestro la rete.

In [ ]:
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_val, y_val), callbacks=[early_stopping])

Ora definisco una funzione per la creazione di grafici della storia del training.

In [ ]:
print(history.history.keys())

def plot_training_history(history):

    # Estrai la loss di training e le due accuracy di validation, una per ogni output
    training_loss = history.history['loss']
    validation_loss = history.history['val_loss']
    val_first_component_class_accuracy = history.history['val_FirstComponentClass_accuracy']
    val_second_component_class_accuracy = history.history['val_SecondComponentClass_accuracy']
    first_component_class_accuracy = history.history['FirstComponentClass_accuracy']
    second_component_class_accuracy = history.history['SecondComponentClass_accuracy']

    avg_val_accuracy = [(first_accuracy + second_accuracy) / 2 for (first_accuracy, second_accuracy) in zip(val_first_component_class_accuracy, val_second_component_class_accuracy)]
    avg_train_accuracy = [(first_accuracy + second_accuracy) / 2 for (first_accuracy, second_accuracy) in zip(first_component_class_accuracy, second_component_class_accuracy)]

    # Crea un grafico
    epochs = range(1, len(avg_val_accuracy) + 1)
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 1)
    plt.plot(epochs, training_loss, label='Training Loss')
    plt.plot(epochs, validation_loss, label='Validation Loss')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, avg_train_accuracy, label='Average Training Accuracy')
    plt.plot(epochs, avg_val_accuracy, label='Average Validation Accuracy')
    plt.title('Average Accuracy')
    plt.xlabel('Epochs')
    plt.legend()

    plt.tight_layout()
    plt.show()

plot_training_history(history)

Definisco due funzioni per la valutazione finale del modello

In [ ]:
def eval_model(model):
  x_test, y_test = next(testgen)
  if isinstance(model, keras.Model):
    guesses = model.predict(x_test)
    correct_guesses_1 = np.argmax(guesses[0], axis=1) == np.argmax(y_test[0], axis=1)
    correct_guesses_2 = np.argmax(guesses[1], axis=1) == np.argmax(y_test[1], axis=1)
  else:
    guesses = model(x_test)
    correct_guesses_1 = guesses[:,0] == np.argmax(y_test[0], axis=1)
    correct_guesses_2 = guesses[:,1] == np.argmax(y_test[1], axis=1)
  return (np.mean(correct_guesses_1) + np.mean(correct_guesses_2)) / 2

def multiple_eval_model(model, repeat_eval = 10):
  eval_results = []
  for i in range(repeat_eval):
    eval_results.append(eval_model(model))
  print("mean accuracy = ", np.mean(eval_results))
  print("standard deviation = ", np.std(eval_results))

Valuto la rete e il modello casuale e confronto le loro prestazioni.
Si può facilmente dimostrare che la media delle accuratezze
calcolata utilizzando la funzione "multiple_eval_model" è un numero
appartenente all'intervallo [0,1]

In [ ]:
print('Valutazione modello casuale')
multiple_eval_model(random_model)

print('Valutazione rete neurale')
multiple_eval_model(model)